In [4]:
# RVFL on Breast Cancer dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load and preprocess Breast Cancer dataset
print("Loading Breast Cancer dataset...")
data = load_breast_cancer()
X = StandardScaler().fit_transform(data.data)
y = data.target
y_onehot = np.eye(2)[y]  # One-hot encoding

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameter settings
hidden_nodes_list = [10, 20, 30]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}

best_config = {}
best_test_acc = 0

print("Starting hyperparameter tuning...\n")

for act_name, act_func in activation_functions.items():
    for n_hidden in hidden_nodes_list:
        input_size = X.shape[1]
        W = np.random.randn(n_hidden, input_size)
        b = np.random.randn(n_hidden)

        def compute_H(X):
            H = act_func(np.dot(X, W.T) + b)
            return np.hstack([H, X])  # Concatenate hidden layer output and input

        # Train
        start_train = time.time()
        H_train = compute_H(X_train)
        output_weights = np.linalg.pinv(H_train) @ y_train
        end_train = time.time()

        # Predict on training data
        y_train_pred = H_train @ output_weights
        train_pred_labels = np.argmax(y_train_pred, axis=1)
        train_true_labels = np.argmax(y_train, axis=1)
        train_acc = accuracy_score(train_true_labels, train_pred_labels)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

        # Predict on test data
        start_test = time.time()
        H_test = compute_H(X_test)
        y_test_pred = H_test @ output_weights
        end_test = time.time()

        test_pred_labels = np.argmax(y_test_pred, axis=1)
        test_true_labels = np.argmax(y_test, axis=1)
        test_acc = accuracy_score(test_true_labels, test_pred_labels)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

        print(f"[{act_name}] Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f} | TrainRMSE={train_rmse:.4f} TestRMSE={test_rmse:.4f}")

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_config = {
                'activation': act_name,
                'n_hidden': n_hidden,
                'train_acc': train_acc,
                'test_acc': test_acc,
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_time': end_train - start_train,
                'test_time': end_test - start_test
            }

# Print best result
print("\nBest Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")


Loading Breast Cancer dataset...
Starting hyperparameter tuning...

[sigmoid] Hidden=10 | TrainAcc=0.9670 TestAcc=0.9649 | TrainRMSE=0.2202 TestRMSE=0.2392
[sigmoid] Hidden=20 | TrainAcc=0.9846 TestAcc=0.9561 | TrainRMSE=0.1994 TestRMSE=0.2359
[sigmoid] Hidden=30 | TrainAcc=0.9648 TestAcc=0.9386 | TrainRMSE=0.1985 TestRMSE=0.2390
[tanh] Hidden=10 | TrainAcc=0.9714 TestAcc=0.9474 | TrainRMSE=0.3168 TestRMSE=0.3432
[tanh] Hidden=20 | TrainAcc=0.9802 TestAcc=0.9474 | TrainRMSE=0.2520 TestRMSE=0.2965
[tanh] Hidden=30 | TrainAcc=0.9802 TestAcc=0.9649 | TrainRMSE=0.2283 TestRMSE=0.2348

Best Configuration:
activation: sigmoid
n_hidden: 10
train_acc: 0.967032967032967
test_acc: 0.9649122807017544
train_rmse: 0.22018349337845936
test_rmse: 0.2391559513473764
train_time: 0.0026183128356933594
test_time: 0.0002033710479736328


In [5]:
# deepRVFL on Breast Cancer dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load Breast Cancer dataset
print("Loading Breast Cancer dataset...")
data = load_breast_cancer()
X = StandardScaler().fit_transform(data.data)
y = data.target
y_onehot = np.eye(2)[y]  # One-hot encoding for binary classification

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameters
depth_list = [2, 3]
hidden_nodes_list = [10, 20]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}
lambda_ridge = 1e-3

best_config = {}
best_test_acc = 0

print("\nStarting Deep RVFL tuning...\n")

for act_name, act_func in activation_functions.items():
    for depth in depth_list:
        for n_hidden in hidden_nodes_list:
            input_size = X.shape[1]
            Ws, bs = [], []
            np.random.seed(42)

            current_input_size = input_size
            for _ in range(depth):
                W = np.random.randn(n_hidden, current_input_size)
                b = np.random.randn(n_hidden)
                Ws.append(W)
                bs.append(b)
                current_input_size += n_hidden

            def compute_deep_H(X_input):
                concat_features = X_input
                for l in range(depth):
                    H = act_func(np.dot(concat_features, Ws[l].T) + bs[l])
                    concat_features = np.hstack([concat_features, H])
                return concat_features

            # Train
            start_train = time.time()
            H_train = compute_deep_H(X_train)
            I = np.identity(H_train.shape[1])
            output_weights = np.linalg.inv(H_train.T @ H_train + lambda_ridge * I) @ H_train.T @ y_train
            end_train = time.time()

            y_train_pred = H_train @ output_weights
            train_true_labels = np.argmax(y_train, axis=1)
            train_pred_labels = np.argmax(y_train_pred, axis=1)
            train_acc = accuracy_score(train_true_labels, train_pred_labels)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

            # Test
            start_test = time.time()
            H_test = compute_deep_H(X_test)
            y_test_pred = H_test @ output_weights
            end_test = time.time()

            test_true_labels = np.argmax(y_test, axis=1)
            test_pred_labels = np.argmax(y_test_pred, axis=1)
            test_acc = accuracy_score(test_true_labels, test_pred_labels)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

            print(f"[{act_name}] Depth={depth} Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f} | TrainRMSE={train_rmse:.4f} TestRMSE={test_rmse:.4f}")

            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_config = {
                    'activation': act_name,
                    'depth': depth,
                    'n_hidden': n_hidden,
                    'train_acc': train_acc,
                    'test_acc': test_acc,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'train_time': end_train - start_train,
                    'test_time': end_test - start_test
                }

# Print best configuration
print("\nBest Deep RVFL Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")


Loading Breast Cancer dataset...

Starting Deep RVFL tuning...

[sigmoid] Depth=2 Hidden=10 | TrainAcc=0.9736 TestAcc=0.9737 | TrainRMSE=0.1952 TestRMSE=0.2103
[sigmoid] Depth=2 Hidden=20 | TrainAcc=0.9780 TestAcc=0.9474 | TrainRMSE=0.1811 TestRMSE=0.2308
[sigmoid] Depth=3 Hidden=10 | TrainAcc=0.9758 TestAcc=0.9561 | TrainRMSE=0.1908 TestRMSE=0.2125
[sigmoid] Depth=3 Hidden=20 | TrainAcc=0.9824 TestAcc=0.9386 | TrainRMSE=0.1683 TestRMSE=0.2317
[tanh] Depth=2 Hidden=10 | TrainAcc=0.9692 TestAcc=0.9561 | TrainRMSE=0.3711 TestRMSE=0.3909
[tanh] Depth=2 Hidden=20 | TrainAcc=0.9780 TestAcc=0.9474 | TrainRMSE=0.2419 TestRMSE=0.3049
[tanh] Depth=3 Hidden=10 | TrainAcc=0.9648 TestAcc=0.9737 | TrainRMSE=0.3357 TestRMSE=0.3499
[tanh] Depth=3 Hidden=20 | TrainAcc=0.9824 TestAcc=0.9474 | TrainRMSE=0.2272 TestRMSE=0.2897

Best Deep RVFL Configuration:
activation: sigmoid
depth: 2
n_hidden: 10
train_acc: 0.9736263736263736
test_acc: 0.9736842105263158
train_rmse: 0.19522344006276332
test_rmse: 0.210

In [6]:
# ensembledeepRVFL on Breast Cancer dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load Breast Cancer dataset
print("Loading Breast Cancer dataset...")
data = load_breast_cancer()
X = StandardScaler().fit_transform(data.data)
y = data.target
y_onehot = np.eye(2)[y]  # One-hot encode labels for binary classification

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameters
depth_list = [2, 3]
hidden_nodes_list = [10, 20]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}
lambda_ridge = 1e-3

best_config = {}
best_test_acc = 0

print("\nStarting ensemble DRVFL tuning...\n")

for act_name, act_func in activation_functions.items():
    for depth in depth_list:
        for n_hidden in hidden_nodes_list:
            input_size = X.shape[1]
            Ws, bs = [], []
            np.random.seed(42)

            current_input_size = input_size
            for _ in range(depth):
                W = np.random.randn(n_hidden, current_input_size)
                b = np.random.randn(n_hidden)
                Ws.append(W)
                bs.append(b)
                current_input_size += n_hidden

            def layer_forward(X_input, W, b):
                H = act_func(np.dot(X_input, W.T) + b)
                return np.hstack([X_input, H])

            # Train
            start_train = time.time()
            layer_outputs_train = []
            current_input = X_train.copy()

            for l in range(depth):
                H = layer_forward(current_input, Ws[l], bs[l])
                current_input = H
                I = np.identity(H.shape[1])
                beta = np.linalg.inv(H.T @ H + lambda_ridge * I) @ H.T @ y_train
                layer_outputs_train.append((H, beta))

            end_train = time.time()

            # Ensemble training prediction
            y_train_preds = [H @ beta for H, beta in layer_outputs_train]
            y_train_ensemble = np.mean(y_train_preds, axis=0)
            train_pred_labels = np.argmax(y_train_ensemble, axis=1)
            train_true_labels = np.argmax(y_train, axis=1)
            train_acc = accuracy_score(train_true_labels, train_pred_labels)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_ensemble))

            # Test
            start_test = time.time()
            layer_outputs_test = []
            current_input = X_test.copy()

            for l in range(depth):
                H_test = layer_forward(current_input, Ws[l], bs[l])
                current_input = H_test
                _, beta = layer_outputs_train[l]
                pred_test = H_test @ beta
                layer_outputs_test.append(pred_test)

            y_test_ensemble = np.mean(layer_outputs_test, axis=0)
            end_test = time.time()

            test_pred_labels = np.argmax(y_test_ensemble, axis=1)
            test_true_labels = np.argmax(y_test, axis=1)
            test_acc = accuracy_score(test_true_labels, test_pred_labels)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_ensemble))

            print(f"[{act_name}] Depth={depth} Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f} | TrainRMSE={train_rmse:.4f} TestRMSE={test_rmse:.4f}")

            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_config = {
                    'activation': act_name,
                    'depth': depth,
                    'n_hidden': n_hidden,
                    'train_acc': train_acc,
                    'test_acc': test_acc,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'train_time': end_train - start_train,
                    'test_time': end_test - start_test
                }

# Print best configuration
print("\nBest Ensemble DRVFL Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")


Loading Breast Cancer dataset...

Starting ensemble DRVFL tuning...

[sigmoid] Depth=2 Hidden=10 | TrainAcc=0.9758 TestAcc=0.9649 | TrainRMSE=0.1990 TestRMSE=0.2140
[sigmoid] Depth=2 Hidden=20 | TrainAcc=0.9780 TestAcc=0.9474 | TrainRMSE=0.1852 TestRMSE=0.2263
[sigmoid] Depth=3 Hidden=10 | TrainAcc=0.9736 TestAcc=0.9649 | TrainRMSE=0.1945 TestRMSE=0.2111
[sigmoid] Depth=3 Hidden=20 | TrainAcc=0.9780 TestAcc=0.9474 | TrainRMSE=0.1760 TestRMSE=0.2227
[tanh] Depth=2 Hidden=10 | TrainAcc=0.9670 TestAcc=0.9649 | TrainRMSE=0.3756 TestRMSE=0.3977
[tanh] Depth=2 Hidden=20 | TrainAcc=0.9802 TestAcc=0.9298 | TrainRMSE=0.2468 TestRMSE=0.3072
[tanh] Depth=3 Hidden=10 | TrainAcc=0.9670 TestAcc=0.9649 | TrainRMSE=0.3540 TestRMSE=0.3709
[tanh] Depth=3 Hidden=20 | TrainAcc=0.9824 TestAcc=0.9386 | TrainRMSE=0.2361 TestRMSE=0.2959

Best Ensemble DRVFL Configuration:
activation: sigmoid
depth: 2
n_hidden: 10
train_acc: 0.9758241758241758
test_acc: 0.9649122807017544
train_rmse: 0.199012804636247
test_rms

In [ ]:
#RVFL on MNIST dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load MNIST dataset
print("Loading MNIST dataset...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X / 255.0  # Normalize pixel values to [0, 1]
y = y.astype(int)
y_onehot = np.eye(10)[y]  # One-hot encode labels

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameter settings
hidden_nodes_list = [10, 20, 30]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}

best_config = {}
best_test_acc = 0

print("Starting hyperparameter tuning...\n")

for act_name, act_func in activation_functions.items():
    for n_hidden in hidden_nodes_list:
        input_size = X.shape[1]
        W = np.random.randn(n_hidden, input_size)
        b = np.random.randn(n_hidden)

        def compute_H(X):
            H = act_func(np.dot(X, W.T) + b)
            return np.hstack([H, X])  # Concatenate hidden layer output and input

        # Train
        start_train = time.time()
        H_train = compute_H(X_train)
        output_weights = np.linalg.pinv(H_train) @ y_train
        end_train = time.time()

        # Predict on training data
        y_train_pred = H_train @ output_weights
        train_pred_labels = np.argmax(y_train_pred, axis=1)
        train_true_labels = np.argmax(y_train, axis=1)
        train_acc = accuracy_score(train_true_labels, train_pred_labels)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

        # Predict on test data
        start_test = time.time()
        H_test = compute_H(X_test)
        y_test_pred = H_test @ output_weights
        end_test = time.time()

        test_pred_labels = np.argmax(y_test_pred, axis=1)
        test_true_labels = np.argmax(y_test, axis=1)
        test_acc = accuracy_score(test_true_labels, test_pred_labels)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

        print(f"[{act_name}] Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f} | TrainRMSE={train_rmse:.4f} TestRMSE={test_rmse:.4f}")

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_config = {
                'activation': act_name,
                'n_hidden': n_hidden,
                'train_acc': train_acc,
                'test_acc': test_acc,
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_time': end_train - start_train,
                'test_time': end_test - start_test
            }

# Print best result
print("\nBest Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")

Loading MNIST dataset...
Starting hyperparameter tuning...

[sigmoid] Hidden=10 | TrainAcc=0.8616 TestAcc=0.8544 | TrainRMSE=0.1941 TestRMSE=0.1979
[sigmoid] Hidden=20 | TrainAcc=0.8666 TestAcc=0.8581 | TrainRMSE=0.1924 TestRMSE=0.1960
[sigmoid] Hidden=30 | TrainAcc=0.8672 TestAcc=0.8589 | TrainRMSE=0.1919 TestRMSE=0.1956
[tanh] Hidden=10 | TrainAcc=0.8568 TestAcc=0.8506 | TrainRMSE=0.1961 TestRMSE=0.1999
[tanh] Hidden=20 | TrainAcc=0.8616 TestAcc=0.8562 | TrainRMSE=0.1942 TestRMSE=0.1979
[tanh] Hidden=30 | TrainAcc=0.8653 TestAcc=0.8586 | TrainRMSE=0.1926 TestRMSE=0.1962

Best Configuration:
activation: sigmoid
n_hidden: 30
train_acc: 0.8672142857142857
test_acc: 0.8589285714285714
train_rmse: 0.1918564836498676
test_rmse: 0.19559669185177567
train_time: 20.80580759048462
test_time: 0.11172723770141602


In [ ]:
#deepRVFL on MNIST dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load MNIST dataset
print("Loading MNIST...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X / 255.0
y = y.astype(int)
y_onehot = np.eye(10)[y]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameters
depth_list = [2, 3]
hidden_nodes_list = [10, 20]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}
lambda_ridge = 1e-3

best_config = {}
best_test_acc = 0

print("\nStarting Deep RVFL tuning...\n")

for act_name, act_func in activation_functions.items():
    for depth in depth_list:
        for n_hidden in hidden_nodes_list:
            input_size = X.shape[1]
            Ws, bs = [], []
            np.random.seed(42)

            # Update input size for each layer
            current_input_size = input_size
            for _ in range(depth):
                W = np.random.randn(n_hidden, current_input_size)
                b = np.random.randn(n_hidden)
                Ws.append(W)
                bs.append(b)
                current_input_size += n_hidden  # Because of concatenation

            def compute_deep_H(X_input):
                concat_features = X_input
                for l in range(depth):
                    H = act_func(np.dot(concat_features, Ws[l].T) + bs[l])
                    concat_features = np.hstack([concat_features, H])
                return concat_features

            # Train
            start_train = time.time()
            H_train = compute_deep_H(X_train)
            I = np.identity(H_train.shape[1])
            output_weights = np.linalg.inv(H_train.T @ H_train + lambda_ridge * I) @ H_train.T @ y_train
            end_train = time.time()

            # Train predictions
            y_train_pred = H_train @ output_weights
            train_true_labels = np.argmax(y_train, axis=1)
            train_pred_labels = np.argmax(y_train_pred, axis=1)
            train_acc = accuracy_score(train_true_labels, train_pred_labels)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

            # Test
            start_test = time.time()
            H_test = compute_deep_H(X_test)
            y_test_pred = H_test @ output_weights
            end_test = time.time()

            test_true_labels = np.argmax(y_test, axis=1)
            test_pred_labels = np.argmax(y_test_pred, axis=1)
            test_acc = accuracy_score(test_true_labels, test_pred_labels)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

            print(f"[{act_name}] Depth={depth} Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f}")

            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_config = {
                    'activation': act_name,
                    'depth': depth,
                    'n_hidden': n_hidden,
                    'train_acc': train_acc,
                    'test_acc': test_acc,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'train_time': end_train - start_train,
                    'test_time': end_test - start_test
                }

# Print best configuration
print("\nBest Deep RVFL Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")


Loading MNIST...

Starting Deep RVFL tuning...

[sigmoid] Depth=2 Hidden=10 | TrainAcc=0.8635 TestAcc=0.8582
[sigmoid] Depth=2 Hidden=20 | TrainAcc=0.8717 TestAcc=0.8672
[sigmoid] Depth=3 Hidden=10 | TrainAcc=0.8664 TestAcc=0.8601
[sigmoid] Depth=3 Hidden=20 | TrainAcc=0.8758 TestAcc=0.8699
[tanh] Depth=2 Hidden=10 | TrainAcc=0.8612 TestAcc=0.8566
[tanh] Depth=2 Hidden=20 | TrainAcc=0.8676 TestAcc=0.8644
[tanh] Depth=3 Hidden=10 | TrainAcc=0.8626 TestAcc=0.8563
[tanh] Depth=3 Hidden=20 | TrainAcc=0.8703 TestAcc=0.8665

Best Deep RVFL Configuration:
activation: sigmoid
depth: 3
n_hidden: 20
train_acc: 0.8757857142857143
test_acc: 0.8698571428571429
train_rmse: 0.18754425184958304
test_rmse: 0.19025904332080093
train_time: 5.361662864685059
test_time: 0.38353872299194336


In [ ]:
# ensembledeepRVFL on MNIST dataset with hyperparameter tuning
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

# Load MNIST
print("Loading MNIST...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
X = X / 255.0
y = y.astype(int)
y_onehot = np.eye(10)[y]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42
)

# Hyperparameters
depth_list = [2, 3]
hidden_nodes_list = [10, 20]
activation_functions = {'sigmoid': sigmoid, 'tanh': tanh}
lambda_ridge = 1e-3

best_config = {}
best_test_acc = 0

print("\nStarting ensemble DRVFL tuning...\n")

for act_name, act_func in activation_functions.items():
    for depth in depth_list:
        for n_hidden in hidden_nodes_list:
            input_size = X.shape[1]
            Ws, bs = [], []
            np.random.seed(42)

            # Dynamically increase input dimension at each layer
            current_input_size = input_size
            for _ in range(depth):
                W = np.random.randn(n_hidden, current_input_size)
                b = np.random.randn(n_hidden)
                Ws.append(W)
                bs.append(b)
                current_input_size += n_hidden

            def layer_forward(X_input, W, b):
                H = act_func(np.dot(X_input, W.T) + b)
                return np.hstack([X_input, H])

            # Train
            start_train = time.time()
            layer_outputs_train = []
            current_input = X_train.copy()

            for l in range(depth):
                H = layer_forward(current_input, Ws[l], bs[l])
                current_input = H
                I = np.identity(H.shape[1])
                beta = np.linalg.inv(H.T @ H + lambda_ridge * I) @ H.T @ y_train
                layer_outputs_train.append((H, beta))

            end_train = time.time()

            # Ensemble prediction on training set
            y_train_preds = [H @ beta for H, beta in layer_outputs_train]
            y_train_ensemble = np.mean(y_train_preds, axis=0)
            train_pred_labels = np.argmax(y_train_ensemble, axis=1)
            train_true_labels = np.argmax(y_train, axis=1)
            train_acc = accuracy_score(train_true_labels, train_pred_labels)
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_ensemble))

            # Test
            start_test = time.time()
            layer_outputs_test = []
            current_input = X_test.copy()

            for l in range(depth):
                H_test = layer_forward(current_input, Ws[l], bs[l])
                current_input = H_test
                _, beta = layer_outputs_train[l]
                pred_test = H_test @ beta
                layer_outputs_test.append(pred_test)

            y_test_ensemble = np.mean(layer_outputs_test, axis=0)
            end_test = time.time()

            test_pred_labels = np.argmax(y_test_ensemble, axis=1)
            test_true_labels = np.argmax(y_test, axis=1)
            test_acc = accuracy_score(test_true_labels, test_pred_labels)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_ensemble))

            print(f"[{act_name}] Depth={depth} Hidden={n_hidden} | TrainAcc={train_acc:.4f} TestAcc={test_acc:.4f}")

            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_config = {
                    'activation': act_name,
                    'depth': depth,
                    'n_hidden': n_hidden,
                    'train_acc': train_acc,
                    'test_acc': test_acc,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'train_time': end_train - start_train,
                    'test_time': end_test - start_test
                }

# Best config
print("\nBest Ensemble DRVFL Configuration:")
for k, v in best_config.items():
    print(f"{k}: {v}")


Loading MNIST...

Starting ensemble DRVFL tuning...

[sigmoid] Depth=2 Hidden=10 | TrainAcc=0.8628 TestAcc=0.8575
[sigmoid] Depth=2 Hidden=20 | TrainAcc=0.8690 TestAcc=0.8645
[sigmoid] Depth=3 Hidden=10 | TrainAcc=0.8643 TestAcc=0.8589
[sigmoid] Depth=3 Hidden=20 | TrainAcc=0.8729 TestAcc=0.8662
[tanh] Depth=2 Hidden=10 | TrainAcc=0.8611 TestAcc=0.8552
[tanh] Depth=2 Hidden=20 | TrainAcc=0.8653 TestAcc=0.8599
[tanh] Depth=3 Hidden=10 | TrainAcc=0.8617 TestAcc=0.8566
[tanh] Depth=3 Hidden=20 | TrainAcc=0.8676 TestAcc=0.8631

Best Ensemble DRVFL Configuration:
activation: sigmoid
depth: 3
n_hidden: 20
train_acc: 0.8728571428571429
test_acc: 0.8662142857142857
train_rmse: 0.1888222209250521
test_rmse: 0.19157156394096436
train_time: 15.812729835510254
test_time: 0.5134727954864502
